In [1]:
import os
os.chdir("clifford-group-equivariant-neural-networks")

# os.chdir('/')
# if not os.path.exists("/clifford-group-equivariant-neural-networks"):
#     !git clone https://github.com/DavidRuhe/clifford-group-equivariant-neural-networks.git

# !git clone https://github.com/DavidRuhe/clifford-group-equivariant-neural-networks.git
# !cd clifford-group-equivariant-neural-networks


In [2]:
import torch
from torch import nn, optim
import torch.nn.functional as F
import torch
from algebra.cliffordalgebra import CliffordAlgebra
from models.modules.linear import MVLinear
from models.modules.gp import SteerableGeometricProductLayer
from models.modules.mvlayernorm import MVLayerNorm
from models.modules.mvsilu import MVSiLU


In [3]:
# Define the metric for 3D space (Euclidean)
metric = [1, 1, 1]
d = len(metric)

# Initialize the Clifford Algebra for 3D
clifford_algebra = CliffordAlgebra(metric)

## DATAPOINTS

Example 1 datapoint

In [4]:
# Example point (x, y, z) and charge (q)
x, y, z, q = 1.0, 2.0, 3.0, 0.5

# Embed the point into Clifford Algebra as a vector
vector = torch.tensor([x, y, z])
vector_in_clifford = clifford_algebra.embed_grade(vector, grade=1)
# Include the charge as a scalar part of the algebra
charge_in_clifford = clifford_algebra.embed_grade(torch.tensor([q]), grade=0)

# Combine the vector and charge into a single multivector
point_with_charge = vector_in_clifford + charge_in_clifford

# The 'point_with_charge' now represents the 3D point and its charge in Clifford space
print(point_with_charge)



tensor([0.5000, 1.0000, 2.0000, 3.0000, 0.0000, 0.0000, 0.0000, 0.0000])


Example multiple datapoints

In [5]:
# Define coordinates and charges for 5 samples
# Coordinates are given as a (5, 3) tensor where each row is a point (x, y, z)
coordinates = torch.tensor([
    [1.0, 2.0, 3.0],
    [4.0, 5.0, 6.0],
    [7.0, 8.0, 9.0],
    [10.0, 11.0, 12.0],
    [13.0, 14.0, 15.0]
])

# Charges are given as a (5, 1) tensor where each row is a charge
charges = torch.tensor([
    [0.5],
    [1.0],
    [1.5],
    [2.0],
    [2.5]
])

# Embed the vectors (coordinates) into Clifford Algebra
vectors_in_clifford = clifford_algebra.embed_grade(coordinates, grade=1)

# Embed the charges as scalar parts of the algebra
charges_in_clifford = clifford_algebra.embed_grade(charges, grade=0)

# Combine the vectors and charges into single multivectors for each sample
points_with_charge = vectors_in_clifford + charges_in_clifford

# 'points_with_charge' now contains the 5 points each with a charge in Clifford space
print(points_with_charge)


tensor([[ 0.5000,  1.0000,  2.0000,  3.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.0000,  4.0000,  5.0000,  6.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.5000,  7.0000,  8.0000,  9.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 2.0000, 10.0000, 11.0000, 12.0000,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 2.5000, 13.0000, 14.0000, 15.0000,  0.0000,  0.0000,  0.0000,  0.0000]])


Example NBODY points, but all features added into 1 clifford vector

In [6]:
# Number of bodies
N = 5  # Example for 5 bodies, can be any number

# Define properties: positions, charges, velocities
positions = torch.randn(N, 3)  # Random 3D positions
charges = torch.randn(N, 1)    # Random charges
velocities = torch.randn(N, 3) # Random 3D velocities

# Embed the properties into Clifford Algebra
positions_in_clifford = clifford_algebra.embed_grade(positions, grade=1)
charges_in_clifford = clifford_algebra.embed_grade(charges, grade=0)
velocities_in_clifford = clifford_algebra.embed_grade(velocities, grade=1)  # Also a vector

# Combine all properties into a single multivector for each body
bodies = positions_in_clifford + charges_in_clifford + velocities_in_clifford

# 'bodies' now contains N multivectors, each representing a body with position, charge, and velocity
print(bodies)
print(bodies.shape)


tensor([[-1.9884, -0.8536, -1.9983,  3.2983,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.7161,  0.3965,  2.0751,  2.3322,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 1.0808, -1.7179,  1.8577,  1.0985,  0.0000,  0.0000,  0.0000,  0.0000],
        [ 0.5275,  0.3335,  0.0614,  1.1787,  0.0000,  0.0000,  0.0000,  0.0000],
        [-0.1833,  0.5944,  0.7600, -0.4432,  0.0000,  0.0000,  0.0000,  0.0000]])
torch.Size([5, 8])


Example full with concat like in the code (seperate clifford vector for all features):

In [7]:
# Number of bodies
N = 5  # Example for 5 bodies, can be any number

# Define properties: positions, charges, velocities
positions = torch.randn(N, 3)  # Random 3D positions
charges = torch.randn(N, 1)    # Random charges
velocities = torch.randn(N, 3) # Random 3D velocities

# Embed the properties into Clifford Algebra
charges_in_clifford = clifford_algebra.embed_grade(charges, grade=0)

# positions_in_clifford = clifford_algebra.embed_grade(positions, grade=1)
# velocities_in_clifford = clifford_algebra.embed_grade(velocities, grade=1)  # Also a vector

xv = torch.stack([positions, velocities], dim=1)
embedded_xv = clifford_algebra.embed(xv, (1, 2, 3))

bodies = torch.cat([charges_in_clifford[:, None], embedded_xv], dim=1)

# (batch_size, feature_size) -> (batch_size, 1, feature_size). The None index is a shorthand for unsqueeze, which adds a singleton dimension at the specified index (here, index 1).
print("Invariants reshaped:", charges_in_clifford[:, None].shape)  # (5, 1, 8)
print("Covariants:", embedded_xv.shape)                    # (5, 2, 8)
print("Concatenated input:", bodies.shape)                 # (5, 3, 8)

# 'bodies' now contains N multivectors, each a sample with with position, charge, and velocity

Invariants reshaped: torch.Size([5, 1, 8])
Covariants: torch.Size([5, 2, 8])
Concatenated input: torch.Size([5, 3, 8])


# EDGES

Now lets look at the edges of the graph as defined above (N=5)

In [8]:
# Edges and edge attributes
# Assuming fully connected graph, every node to every other node
edges = torch.combinations(torch.arange(N), r=2)

print("Edges:")
print(edges)


Edges:
tensor([[0, 1],
        [0, 2],
        [0, 3],
        [0, 4],
        [1, 2],
        [1, 3],
        [1, 4],
        [2, 3],
        [2, 4],
        [3, 4]])


Lets define the edge attributes for now, just take distance

In [9]:
distance = torch.norm(positions[edges[:, 0]] - positions[edges[:, 1]], dim=1).unsqueeze(1)
edge_attr = clifford_algebra.embed_grade(distance, grade=0)  # Embedding distance as an edge attribute

print("Edge Attributes (Distances):")
print(edge_attr)
print(edge_attr.shape)


Edge Attributes (Distances):
tensor([[2.6651, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.7239, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [3.8979, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [3.5068, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [2.5543, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.6960, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.3947, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [3.0491, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [3.0682, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [1.9414, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000]])
torch.Size([10, 8])


Now, lets imagine we do not have edge attributes, how would we calculate?

In [10]:
edge_attributes = []
for edge in edges:
    node1, node2 = edge

    # difference between node features
    node_i_features = bodies[node1]
    node_j_features = bodies[node2]
    difference = node_i_features - node_j_features

    # geom product between node features
    geom_product = clifford_algebra.geometric_product(node_i_features, node_j_features)

    # now just add the 2 to get the "representation of the edge"
    edge_attributes.append(difference + geom_product)

edge_attributes = torch.stack(edge_attributes)  # Stack all edge attributes into a tensor

print("Edge Attributes (Difference + Geom product):")
print(edge_attributes.shape)

Edge Attributes (Difference + Geom product):
torch.Size([10, 3, 8])


Lets transform the nodes

In [11]:
in_features = bodies.shape[1] # 3
hidden_features = 2

# SUBSPACES??!!
node_to_hidden = MVLinear(clifford_algebra, in_features, hidden_features, subspaces=True)

In [12]:
h_bodies = node_to_hidden(bodies)
print(h_bodies.shape)

torch.Size([5, 2, 8])


And the edges

In [13]:
in_features = edge_attributes.shape[1] # 3
hidden_features = 3

# SUBSPACES??
edge_to_hidden = MVLinear(clifford_algebra, in_features, hidden_features, subspaces=True)

In [14]:
h_edges = edge_to_hidden(edge_attributes)
print(h_edges.shape)

torch.Size([10, 3, 8])


## now that that is out of the way, lets create some functions to streamline everything

In [15]:
def make_edge_attr(node_features, edges, batch_size):
    edge_attributes = []

    total_number_edges = edges[0].shape[0]

    # Loop over all edges
    for i in range(total_number_edges):

        node1 = edges[0][i]
        node2 = edges[1][i]

        # difference between node features
        node_i_features = node_features[node1]  # [#features(charge, loc, vel), dim]
        node_j_features = node_features[node2]  # [#features(charge, loc, vel), dim]
        difference = node_i_features - node_j_features

        # geom product between node features
        geom_product = clifford_algebra.geometric_product(node_i_features, node_j_features)

        # now just add the 2 to get the "representation of the edge"
        # Stack all
        edge_representation = torch.cat((difference, geom_product), dim=0)
        edge_attributes.append(edge_representation)

    edge_attributes = torch.stack(edge_attributes)
    return edge_attributes


In [16]:
def embed_nbody_graphs(batch):
    loc, vel, edge_attr, charges, loc_end, edges = batch

    batch_size, n_nodes, _ = loc.size()


    # put the mean of the pointcloud on the origin (WHY?)
    loc_mean = loc - loc.mean(dim=1, keepdim=True)     # [batch, nodes, dim]


    # ALL THESE do [batch, nodes, dim] ->>> [batch * nodes, dim] BUT WHYYY
    loc_mean = loc_mean.float().view(-1, *loc_mean.shape[2:])
    loc = loc.float().view(-1, *loc.shape[2:])
    vel = vel.float().view(-1, *vel.shape[2:])
    edge_attr = edge_attr.float().view(-1, *edge_attr.shape[2:])
    charges = charges.float().view(-1, *charges.shape[2:])
    loc_end = loc_end.float().view(-1, *loc_end.shape[2:])

    invariants = charges
    invariants = clifford_algebra.embed(invariants, (0,))
    charges_in_clifford = invariants

    xv = torch.stack([loc_mean, vel], dim=1) # now of the shape [batch * nodes, 2 (because its loc mean as well as vel), dim]
    covariants = clifford_algebra.embed(xv, (1, 2, 3))
    pos_vel_in_clifford = covariants


    # the [:, none] adds a dimension to invariants: [500, 8] to 500, 1, 8]) so they can be concatinated
    nodes_in_clifford = torch.cat([invariants[:, None], covariants], dim=1) # [batch * nodes, #features(charge, loc, vel), dim]
    # print('nodes_in_clifford: [batch * nodes, #features(charge, loc, vel), dim]', nodes_in_clifford.shape)

    # till now: edges [batch, 2, n_nodes] so per batch, first row is starting nodes, second row is finishing nodes.
    # Batch 1:  [[0, 1, 2, 3, 4],  # Start nodes
    #            [1, 2, 3, 4, 0]]  # End nodes

    # Batch 2:  [[0, 1, 2, 3, 4],  # Start nodes
    #            [1, 2, 3, 4, 0]]  # End nodes

    # Batch 3:  [[0, 1, 2, 3, 4],  # Start nodes
    #            [1, 2, 3, 4, 0]]  # End nodes


    # THEIR CODE, IDK IF NEEDED
    batch_index = torch.arange(batch_size, device=loc_mean.device) # torch.arange(batch_size) generates a tensor from 0 to batch_size - 1, creating a sequence that represents each graph in the batch. If batch_size is 3, this tensor will be [0, 1, 2]
    edges = edges + n_nodes * batch_index[:, None, None] # creates separate edge number for every graph. so if edge for graph 1 is between 3 and 4, graph 2 will be between 8 and 9 (if n_nodes = 5)
    edges = tuple(edges.transpose(0, 1).flatten(1)) # where the first element of the tuple contains all start nodes and the second contains all end nodes for edges across the entire batch. ([edges*batch], [edges*batch])

    extra_edge_attr_clifford = make_edge_attr(nodes_in_clifford, edges, batch_size) # [batch*edges, #numfeatures, difference + geomprod, dim]

    orig_edge_attr_clifford = clifford_algebra.embed(edge_attr[..., None], (0,)) # now [batch * edges, 1, dim]

    return nodes_in_clifford, extra_edge_attr_clifford, orig_edge_attr_clifford


In [17]:
def generate_mock_batch(batch_size):
    """
    Generate a mock batch of data with specified shapes.

    Parameters:
    - batch_size (int): The size of the batch to generate.

    Returns:
    - A tuple containing tensors for loc_frame_0, vel_frame_0, edge_attr, charges, loc_frame_T, and edges.
    """
    # Constants
    n_nodes = 5  # Number of nodes
    n_features = 3  # Number of spatial features (e.g., x, y, z)
    n_edges = 20  # Number of edges
    n_edge_features = 1  # Number of features per edge

    # Generate data
    loc_frame_0 = torch.rand(batch_size, n_nodes, n_features)
    vel_frame_0 = torch.rand(batch_size, n_nodes, n_features)
    edge_attr = torch.rand(batch_size, n_edges, n_edge_features)
    charges = torch.rand(batch_size, n_nodes, 1)
    loc_frame_T = torch.rand(batch_size, n_nodes, n_features)

    # Generate edges indices
    # For simplicity, assuming all batches share the same structure of graph
    rows = torch.randint(0, n_nodes, (n_edges,))
    cols = torch.randint(0, n_nodes, (n_edges,))
    edges = torch.stack([rows, cols], dim=0).repeat(batch_size, 1, 1)  # Repeat the edge structure across the batch

    return loc_frame_0, vel_frame_0, edge_attr, charges, loc_frame_T, edges

In [18]:
# Example of using the function with a batch size of 69
batch_size = 100
data = generate_mock_batch(batch_size)
# n_nodes = 5  # Number of nodes
# n_features = 3  # Number of spatial features (e.g., x, y, z)
# n_edges = 20  # Number of edges
# n_edge_features = 1  # Number of features per edge

# Print the shapes of each element in the data tuple
data_shapes = [d.shape for d in data]
# print("Shape of loc_frame_0:", data_shapes[0])
# print("Shape of vel_frame_0:", data_shapes[1])
# print("Shape of edge_attr:", data_shapes[2])
# print("Shape of charges:", data_shapes[3])
# print("Shape of loc_frame_T:", data_shapes[4])
# print("Shape of edges:", data_shapes[5])

In [19]:
components = embed_nbody_graphs(data)
nodes_in_clifford, extra_edge_attr_clifford, orig_edge_attr_clifford = components
edges_in_clifford = torch.cat((extra_edge_attr_clifford, orig_edge_attr_clifford), dim=1)

# This MVLinear should be somewhere else! HOW DOES THIS WORK PRECICELY

edge_to_node_shape = MVLinear(clifford_algebra, edges_in_clifford.shape[1] , nodes_in_clifford.shape[1], subspaces=True)

fully_embedded_edges = edge_to_node_shape(edges_in_clifford)


# Adding a binary feature
# Nodes get a [1, 0] and edges get a [0, 1]
node_marker = torch.zeros(500, 1, 8)
edge_marker = torch.ones(2000, 1, 8)

# Concatenate this new feature
nodes_encoded = torch.cat((nodes_in_clifford, node_marker), dim=1)  # New shape [500, 4, 8]
edges_encoded = torch.cat((fully_embedded_edges, edge_marker), dim=1)  # New shape [2000, 4, 8]

# Reshape to split according to graphs
nodes = nodes_encoded.view(100, 5, 4, 8)   # Shape [100, 5, 3, 9]
edges = edges_encoded.view(100, 20, 4, 8)  # Shape [100, 20, 3, 9]

# Initialize a list to hold the combined tensors for each graph
combined = []

# Concatenate nodes and edges for each graph
for i in range(100):
    combined_graph = torch.cat((nodes[i], edges[i]), dim=0)  # Shape [25, 4, 8]
    combined.append(combined_graph)

combined = torch.stack(combined, dim=0)  # Shape [100, 25, 4, 8]
final_tensor = combined.view(2500, 4, 8)  # Shape [2500, 4, 8]

print('combined:', final_tensor.shape)


# print(final_tensor)

combined: torch.Size([2500, 4, 8])


code for learnable positional encodings:

In [20]:
import torch
from torch import nn
import torch.nn.functional as F

class GeometricAlgebraAttention(nn.Module):
    def __init__(self, algebra, features, heads=8):
        super().__init__()
        self.algebra = algebra
        self.heads = heads
        self.features_per_head = features // heads
        
        assert features % heads == 0, "Features must be divisible by heads"
        
        self.query = MVLinear(self.algebra, features, features)
        self.key = MVLinear(self.algebra, features, features)
        self.value = MVLinear(self.algebra, features, features)
        
        self.unifyheads = MVLinear(self.algebra, features, features)

    def forward(self, x):
        b, t, f = x.size()
        print(f"Input size: {x.size()}")

        queries = self.query(x).view(b, t, self.heads, self.features_per_head)
        print(f"Queries size: {queries.size()}")
        keys = self.key(x).view(b, t, self.heads, self.features_per_head)
        print(f"Keys size: {keys.size()}")
        values = self.value(x).view(b, t, self.heads, self.features_per_head)
        print(f"Values size: {values.size()}")
        
        keys = keys.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        queries = queries.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        values = values.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        
        scores = torch.matmul(queries, keys.transpose(1, 2)) / (self.features_per_head ** 0.5)
        scores = F.softmax(scores, dim=2)
        
        combined = torch.matmul(scores, values)
        combined = combined.view(b, self.heads, t, self.features_per_head).transpose(1, 2).contiguous()
        combined = combined.view(b, t, f)
        
        return self.unifyheads(combined)

class NBodyCGGNN(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_heads=8, n_layers=3):
        super().__init__()
        self.algebra = algebra
        self.embedding = MVLinear(self.algebra, in_features, hidden_features)

        self.layers = nn.ModuleList([
            GeometricAlgebraAttention(self.algebra, hidden_features, heads=n_heads)
            for _ in range(n_layers)
        ])
        
        self.projection = MVLinear(self.algebra, hidden_features, out_features)

    def forward(self, x):
        x = self.embedding(x)
        
        for layer in self.layers:
            x = layer(x)
        
        return self.projection(x)

In [21]:
class GeometricAlgebraAttention(nn.Module):
    def __init__(self, algebra, features, heads=8):
        super().__init__()
        self.algebra = algebra
        self.heads = heads
        self.features_per_head = features // heads
        assert features % heads == 0, "Features must be divisible by heads"
        
        self.query = MVLinear(self.algebra, features, features)
        self.key = MVLinear(self.algebra, features, features)
        self.value = MVLinear(self.algebra, features, features)
        
        self.unifyheads = MVLinear(self.algebra, features, features)

    def forward(self, x):
        b, t, f = x.size()
        print(f"Input size before MVLinear: {x.size()}")  # Check input size

        queries = self.query(x)
        keys = self.key(x)
        values = self.value(x)
        print(f"Output size from MVLinear (queries): {queries.size()}")  # Check output size

        queries = queries.view(b, t, self.heads, self.features_per_head)
        keys = keys.view(b, t, self.heads, self.features_per_head)
        values = values.view(b, t, self.heads, self.features_per_head)

        keys = keys.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        queries = queries.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        values = values.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        
        scores = torch.matmul(queries, keys.transpose(1, 2)) / (self.features_per_head ** 0.5)
        scores = F.softmax(scores, dim=2)
        
        combined = torch.matmul(scores, values)
        combined = combined.view(b, self.heads, t, self.features_per_head).transpose(1, 2).contiguous()
        combined = combined.view(b, t, f)
        
        return self.unifyheads(combined)
        

class NBodyCGGNN(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_heads=8, n_layers=3):
        super().__init__()
        self.algebra = algebra
        self.embedding = MVLinear(self.algebra, in_features, hidden_features)

        self.layers = nn.ModuleList([
            GeometricAlgebraAttention(self.algebra, hidden_features, heads=n_heads)
            for _ in range(n_layers)
        ])
        
        self.projection = MVLinear(self.algebra, hidden_features, out_features)

    def forward(self, x):
        x = self.embedding(x)
        
        for layer in self.layers:
            x = layer(x)
        
        return self.projection(x)


In [22]:
import torch
from torch import nn, optim
from tqdm import tqdm
import torch.nn.functional as F

# Assuming the necessary components are defined as follows:
from algebra.cliffordalgebra import CliffordAlgebra
from models.modules.linear import MVLinear

class GeometricAlgebraAttention(nn.Module):
    def __init__(self, algebra, features, heads=8):
        super().__init__()
        self.algebra = algebra
        self.heads = heads
        self.features_per_head = features // heads

        self.query = MVLinear(self.algebra, features, features)
        self.key = MVLinear(self.algebra, features, features)
        self.value = MVLinear(self.algebra, features, features)

        self.unifyheads = MVLinear(self.algebra, features, features)

    def forward(self, x):
        b, t, f = x.size()

        queries = self.query(x).view(b, t, self.heads, self.features_per_head)
        keys = self.key(x).view(b, t, self.heads, self.features_per_head)
        values = self.value(x).view(b, t, self.heads, self.features_per_head)

        keys = keys.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        queries = queries.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        values = values.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)

        scores = torch.matmul(queries, keys.transpose(1, 2)) / (self.features_per_head ** 0.5)
        scores = F.softmax(scores, dim=2)

        combined = torch.matmul(scores, values)
        combined = combined.view(b, self.heads, t, self.features_per_head).transpose(1, 2).contiguous()
        combined = combined.view(b, t, f)

        return self.unifyheads(combined)

class NBodyGAT(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_heads=8, n_layers=3):
        super().__init__()
        self.algebra = algebra
        self.embedding = MVLinear(self.algebra, in_features, hidden_features)

        self.layers = nn.ModuleList([
            GeometricAlgebraAttention(self.algebra, hidden_features, heads=n_heads)
            for _ in range(n_layers)
        ])

        self.projection = MVLinear(self.algebra, hidden_features, out_features)

    def forward(self, x):
        x = self.embedding(x)

        for layer in self.layers:
            x = layer(x)

        return self.projection(x)

def embed_nbody_graphs(batch):
    loc, vel, edge_attr, charges, loc_end, edges = batch

    batch_size, n_nodes, _ = loc.size()

    loc_mean = loc - loc.mean(dim=1, keepdim=True)     # [batch, nodes, dim]

    loc_mean = loc_mean.float().view(-1, *loc_mean.shape[2:])
    loc = loc.float().view(-1, *loc.shape[2:])
    vel = vel.float().view(-1, *vel.shape[2:])
    edge_attr = edge_attr.float().view(-1, *edge_attr.shape[2:])
    charges = charges.float().view(-1, *charges.shape[2:])
    loc_end = loc_end.float().view(-1, *loc_end.shape[2:])

    invariants = charges
    invariants = clifford_algebra.embed(invariants, (0,))
    charges_in_clifford = invariants

    xv = torch.stack([loc_mean, vel], dim=1)
    covariants = clifford_algebra.embed(xv, (1, 2, 3))
    pos_vel_in_clifford = covariants

    nodes_in_clifford = torch.cat([invariants[:, None], covariants], dim=1)

    batch_index = torch.arange(batch_size, device=loc_mean.device)
    edges = edges + n_nodes * batch_index[:, None, None]
    edges = tuple(edges.transpose(0, 1).flatten(1))

    edge_attributes = []
    for i in range(edges[0].shape[0]):
        node1 = edges[0][i]
        node2 = edges[1][i]
        node_i_features = nodes_in_clifford[node1]
        node_j_features = nodes_in_clifford[node2]
        difference = node_i_features - node_j_features
        geom_product = clifford_algebra.geometric_product(node_i_features, node_j_features)
        edge_representation = torch.cat((difference, geom_product), dim=0)
        edge_attributes.append(edge_representation)

    extra_edge_attr_clifford = torch.stack(edge_attributes)
    orig_edge_attr_clifford = clifford_algebra.embed(edge_attr[..., None], (0,))

    return nodes_in_clifford, extra_edge_attr_clifford, orig_edge_attr_clifford

def generate_mock_batch(batch_size):
    n_nodes = 5
    n_features = 3
    n_edges = 20
    n_edge_features = 1

    loc_frame_0 = torch.rand(batch_size, n_nodes, n_features)
    vel_frame_0 = torch.rand(batch_size, n_nodes, n_features)
    edge_attr = torch.rand(batch_size, n_edges, n_edge_features)
    charges = torch.rand(batch_size, n_nodes, 1)
    loc_frame_T = torch.rand(batch_size, n_nodes, n_features)

    rows = torch.randint(0, n_nodes, (n_edges,))
    cols = torch.randint(0, n_nodes, (n_edges,))
    edges = torch.stack([rows, cols], dim=0).repeat(batch_size, 1, 1)

    return loc_frame_0, vel_frame_0, edge_attr, charges, loc_frame_T, edges

# Generate mock data
batch_size = 100
data = generate_mock_batch(batch_size)
num_epochs = 10

# Initialize the Clifford Algebra and model
clifford_algebra = CliffordAlgebra((1.0, 1.0, 1.0))
model = NBodyGAT(clifford_algebra, in_features=4, hidden_features=8, out_features=1)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0
    for i in range(batch_size):
        batch = data[i]  # Assuming data is a list of batches

        loc, vel, edge_attr, charges, loc_end, edges = batch

        # Prepare the input features
        nodes_in_clifford, extra_edge_attr_clifford, orig_edge_attr_clifford = embed_nbody_graphs(batch)
        edges_in_clifford = torch.cat((extra_edge_attr_clifford, orig_edge_attr_clifford), dim=1)

        edge_to_node_shape = MVLinear(clifford_algebra, edges_in_clifford.shape[1], nodes_in_clifford.shape[1], subspaces=True)
        fully_embedded_edges = edge_to_node_shape(edges_in_clifford)

        # Adding a binary feature
        node_marker = torch.zeros(nodes_in_clifford.size(0), 1, nodes_in_clifford.size(2))
        edge_marker = torch.ones(fully_embedded_edges.size(0), 1, fully_embedded_edges.size(2))

        # Concatenate this new feature
        nodes_encoded = torch.cat((nodes_in_clifford, node_marker), dim=1)
        edges_encoded = torch.cat((fully_embedded_edges, edge_marker), dim=1)

        # Reshape to split according to graphs
        nodes = nodes_encoded.view(batch_size, 5, 4, 8)
        edges = edges_encoded.view(batch_size, 20, 4, 8)

        combined = []
        for j in range(batch_size):
            combined_graph = torch.cat((nodes[j], edges[j]), dim=0)
            combined.append(combined_graph)

        combined = torch.stack(combined, dim=0)
        final_tensor = combined.view(-1, 4, 8)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(final_tensor)

        # Assuming targets are in loc_end
        targets = loc_end.view(-1, 1)

        # Compute loss
        loss = criterion(output, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Accumulate loss for reporting
        epoch_loss += loss.item()

    # Print the average loss for this epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/batch_size}')

# Save the trained model
torch.save(model.state_dict(), 'nbody_model.pth')



  0%|          | 0/10 [00:00<?, ?it/s]


ValueError: too many values to unpack (expected 6)

In [ ]:
import torch
from torch import nn, optim
from tqdm import tqdm
import torch.nn.functional as F

# Assuming the necessary components are defined as follows:
from algebra.cliffordalgebra import CliffordAlgebra
from models.modules.linear import MVLinear

class GeometricAlgebraAttention(nn.Module):
    def __init__(self, algebra, features, heads=8):
        super().__init__()
        self.algebra = algebra
        self.heads = heads
        self.features_per_head = features // heads

        self.query = MVLinear(self.algebra, features, features)
        self.key = MVLinear(self.algebra, features, features)
        self.value = MVLinear(self.algebra, features, features)

        self.unifyheads = MVLinear(self.algebra, features, features)

    def forward(self, x):
        b, t, f = x.size()

        queries = self.query(x).view(b, t, self.heads, self.features_per_head)
        keys = self.key(x).view(b, t, self.heads, self.features_per_head)
        values = self.value(x).view(b, t, self.heads, self.features_per_head)

        keys = keys.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        queries = queries.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)
        values = values.transpose(1, 2).contiguous().view(b * self.heads, t, self.features_per_head)

        scores = torch.matmul(queries, keys.transpose(1, 2)) / (self.features_per_head ** 0.5)
        scores = F.softmax(scores, dim=2)

        combined = torch.matmul(scores, values)
        combined = combined.view(b, self.heads, t, self.features_per_head).transpose(1, 2).contiguous()
        combined = combined.view(b, t, f)

        return self.unifyheads(combined)

class NBodyGAT(nn.Module):
    def __init__(self, algebra, in_features, hidden_features, out_features, n_heads=8, n_layers=3):
        super().__init__()
        self.algebra = algebra
        self.embedding = MVLinear(self.algebra, in_features, hidden_features)

        self.layers = nn.ModuleList([
            GeometricAlgebraAttention(self.algebra, hidden_features, heads=n_heads)
            for _ in range(n_layers)
        ])

        self.projection = MVLinear(self.algebra, hidden_features, out_features)

    def forward(self, x):
        x = self.embedding(x)

        for layer in self.layers:
            x = layer(x)

        return self.projection(x)

def embed_nbody_graphs(batch):
    loc, vel, edge_attr, charges, loc_end, edges = batch

    batch_size, n_nodes, _ = loc.size()

    loc_mean = loc - loc.mean(dim=1, keepdim=True)     # [batch, nodes, dim]

    loc_mean = loc_mean.float().view(-1, *loc_mean.shape[2:])
    loc = loc.float().view(-1, *loc.shape[2:])
    vel = vel.float().view(-1, *vel.shape[2:])
    edge_attr = edge_attr.float().view(-1, *edge_attr.shape[2:])
    charges = charges.float().view(-1, *charges.shape[2:])
    loc_end = loc_end.float().view(-1, *loc_end.shape[2:])

    invariants = charges
    invariants = clifford_algebra.embed(invariants, (0,))
    charges_in_clifford = invariants

    xv = torch.stack([loc_mean, vel], dim=1)
    covariants = clifford_algebra.embed(xv, (1, 2, 3))
    pos_vel_in_clifford = covariants

    nodes_in_clifford = torch.cat([invariants[:, None], covariants], dim=1)

    batch_index = torch.arange(batch_size, device=loc_mean.device)
    edges = edges + n_nodes * batch_index[:, None, None]
    edges = tuple(edges.transpose(0, 1).flatten(1))

    edge_attributes = []
    for i in range(edges[0].shape[0]):
        node1 = edges[0][i]
        node2 = edges[1][i]
        node_i_features = nodes_in_clifford[node1]
        node_j_features = nodes_in_clifford[node2]
        difference = node_i_features - node_j_features
        geom_product = clifford_algebra.geometric_product(node_i_features, node_j_features)
        edge_representation = torch.cat((difference, geom_product), dim=0)
        edge_attributes.append(edge_representation)

    extra_edge_attr_clifford = torch.stack(edge_attributes)
    orig_edge_attr_clifford = clifford_algebra.embed(edge_attr[..., None], (0,))

    return nodes_in_clifford, extra_edge_attr_clifford, orig_edge_attr_clifford

def generate_mock_batch(batch_size):
    n_nodes = 5
    n_features = 3
    n_edges = 20
    n_edge_features = 1

    loc_frame_0 = torch.rand(batch_size, n_nodes, n_features)
    vel_frame_0 = torch.rand(batch_size, n_nodes, n_features)
    edge_attr = torch.rand(batch_size, n_edges, n_edge_features)
    charges = torch.rand(batch_size, n_nodes, 1)
    loc_frame_T = torch.rand(batch_size, n_nodes, n_features)

    rows = torch.randint(0, n_nodes, (n_edges,))
    cols = torch.randint(0, n_nodes, (n_edges,))
    edges = torch.stack([rows, cols], dim=0).repeat(batch_size, 1, 1)

    return loc_frame_0, vel_frame_0, edge_attr, charges, loc_frame_T, edges

# Generate mock data
batch_size = 100
data = generate_mock_batch(batch_size)
num_epochs = 10

# Initialize the Clifford Algebra and model
clifford_algebra = CliffordAlgebra((1.0, 1.0, 1.0))
model = NBodyGAT(clifford_algebra, in_features=4, hidden_features=8, out_features=1)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Check the structure of the data
for i, batch in enumerate(data):
    print(f"Batch {i}: {type(batch)}, shape: {batch.shape if isinstance(batch, torch.Tensor) else 'Not a tensor'}")

# Training loop
for epoch in tqdm(range(num_epochs)):
    epoch_loss = 0

    # Prepare the input features from the data
    loc, vel, edge_attr, charges, loc_end, edges = data

    for i in range(batch_size):
        batch = (loc[i], vel[i], edge_attr[i], charges[i], loc_end[i], edges[:, :, i])  # Extract batch data

        # Unpack batch
        loc, vel, edge_attr, charges, loc_end, edges = batch

        nodes_in_clifford, extra_edge_attr_clifford, orig_edge_attr_clifford = embed_nbody_graphs(batch)
        edges_in_clifford = torch.cat((extra_edge_attr_clifford, orig_edge_attr_clifford), dim=1)

        edge_to_node_shape = MVLinear(clifford_algebra, edges_in_clifford.shape[1], nodes_in_clifford.shape[1], subspaces=True)
        fully_embedded_edges = edge_to_node_shape(edges_in_clifford)

        # Adding a binary feature
        node_marker = torch.zeros(nodes_in_clifford.size(0), 1, nodes_in_clifford.size(2))
        edge_marker = torch.ones(fully_embedded_edges.size(0), 1, fully_embedded_edges.size(2))

        # Concatenate this new feature
        nodes_encoded = torch.cat((nodes_in_clifford, node_marker), dim=1)
        edges_encoded = torch.cat((fully_embedded_edges, edge_marker), dim=1)

        # Reshape to split according to graphs
        nodes = nodes_encoded.view(batch_size, 5, 4, 8)
        edges = edges_encoded.view(batch_size, 20, 4, 8)

        combined = []
        for j in range(batch_size):
            combined_graph = torch.cat((nodes[j], edges[j]), dim=0)
            combined.append(combined_graph)

        combined = torch.stack(combined, dim=0)
        final_tensor = combined.view(-1, 4, 8)

        # Zero the parameter gradients
        optimizer.zero_grad()

        # Forward pass
        output = model(final_tensor)

        # Assuming targets are in loc_end
        targets = loc_end.view(-1, 1)

        # Compute loss
        loss = criterion(output, targets)

        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Accumulate loss for reporting
        epoch_loss += loss.item()

    # Print the average loss for this epoch
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss/batch_size}')

# Save the


In [ ]:
model = NBodyCGGNN(clifford_algebra, in_features=4, hidden_features=8, out_features=1)  # Example with hidden_features=8
final_tensor_reshaped = final_tensor.view(-1, 4, 8)  # Ensure it matches the total elements
output = model(final_tensor_reshaped)
print(output.shape)
print(output)

In [ ]:
clifford_algebra = CliffordAlgebra((1.0, 1.0, 1.0))
model = NBodyCGGNN(clifford_algebra, in_features=4, hidden_features=32, out_features=1)  # Changed hidden_features to 32
final_tensor_reshaped = final_tensor.view(-1, 4, 8)
output = model(final_tensor_reshaped)
print(output.shape)
print(output)

model = NBodyCGGNN(clifford_algebra, in_features=4, hidden_features=8, out_features=1)  # Example with hidden_features=8
final_tensor_reshaped = final_tensor.view(-1, 4, 8)  # Ensure it matches the total elements
output = model(final_tensor_reshaped)
print(output.shape)
print(output)